# SH_neutron_reco_performance

This notebook analyzes neutron reconstruction performance in Solid Hydrogen events using ROOT (PyROOT) and RDataFrame.

## Overview
It loads reconstructed event data from a ROOT file, applies physics filters (volume, reconstruction quality, track topology), and generates comprehensive performance histograms and plots including:
- Neutron momentum and energy residuals (for signal and background events)
- Neutron momentum resolution components (Px, Py, Pz)
- ECAL cluster spatial distributions
- Kinematic prediction accuracy in space (predicted vs detected positions)
- ECAL cluster position and timing resolutions
- Neutron kinematic prediction residuals in time
- Signal/background separation metrics based on spatial and temporal correlations

Results are saved as PDF plots and histograms written to a ROOT output file.

## Usage
Set the input ROOT file path and output directory in the first code cell, then run cells sequentially.

## Dependencies
- ROOT (with PyROOT support)
- Local helper module: `df_functions_utils.py` (provides dataframe extension and filtering functions)


In [ ]:
import ROOT
from ROOT import TCanvas
from ROOT import TLegend
from ROOT import RDataFrame
from ROOT import kRed, kBlue
import df_functions_utils as df_utils
import numpy as np

df = RDataFrame("Rough_tree", "/storage/gpfs_data/neutrino/users/battisti/hydrogen_analysis_tests/Solid_Hydrogen2/Snap_file_processing/Processed_tree.root")
Out_dir = "Plots/SH_neutron_reco_performance/"

In [ ]:
### Uncoment for fast testing
df = df.Range(1000)

### Generating output file

In [ ]:
f_out = ROOT.TFile(Out_dir +"SH_neutron_reco_performance.root", "RECREATE")

# Generating df with additional columns

### List of dataframes:
- df = contains Rough_tree with all its columns
- df_extend =  df + additional columns needed in the analysis 
- df_flag = df_extended + flags that identify the filters  

### Filters flags for df_flag:
- inside the STT volume (volume_filter)
- successufull reconstruction (reco_filter)
- one track (track_filter)
- neutron space correspondence (space_filter, done with clusters) (spaceTRUE_filter, done with hit segment starts)
- neutron time correspondence (time_filter, done with clusters) (timeTRUE_filter, done with hit segment starts)
- events CC-QE with antimuon and neutron in the final state, on H (true_filter) 

In [ ]:
df_extended = df_utils.df_extend(df)
df_flag = df_utils.df_addflags(df_extended)
df_ECAL = df_utils.df_add_ECAL_pred(df_flag)

# Plotting

### Neutron momentum/energy residuals for signal/background events

In [ ]:
df_neutron = df_ECAL.Filter("volume_flag == 1 && reco_flag == 1")


In [ ]:
### Neutron momentum resolution components for true signal events
c = TCanvas("c", " ", 1500, 600)
c.Divide(3,1)
ROOT.gStyle.SetOptStat(0)

max = 1.0
min = -1.0

h_res_n_pxS = df_neutron.Filter("true_flag == 1").Histo1D(("h_res_n_pxS", "res n pxS  ", 100, min, max),"res_pn_x")
h_res_n_pyS = df_neutron.Filter("true_flag == 1").Histo1D(("h_res_n_pyS", "res n pyS ", 100, min, max),"res_pn_y")
h_res_n_pzS = df_neutron.Filter("true_flag == 1").Histo1D(("h_res_n_pzS", "res_n_pzS  ", 100, min, max),"res_pn_z")

c.cd(1)
ROOT.gPad.SetLeftMargin(0.15)
ROOT.gPad.SetRightMargin(0.04)
h_res_n_pxS.GetXaxis().SetTitle("#Delta Px/Px")
h_res_n_pxS.GetYaxis().SetTitle("Entries")
h_res_n_pxS.Draw()

c.cd(2)
ROOT.gPad.SetLeftMargin(0.15)
ROOT.gPad.SetRightMargin(0.04)
h_res_n_pyS.GetXaxis().SetTitle("#Delta Py/Py")
h_res_n_pyS.GetYaxis().SetTitle("Entries")

h_res_n_pyS.Draw()

c.cd(3)
ROOT.gPad.SetLeftMargin(0.15)
ROOT.gPad.SetRightMargin(0.04)
h_res_n_pzS.GetXaxis().SetTitle("#Delta Pz/Pz")
h_res_n_pzS.GetYaxis().SetTitle("Entries")

h_res_n_pzS.Draw()

c.Draw()
c.SaveAs(Out_dir + "neutron_p_res_components_recogood_topologygood.pdf")

f_out.cd()
h_res_n_pxS.GetValue().Write()
h_res_n_pyS.GetValue().Write() 
h_res_n_pzS.GetValue().Write() 

In [ ]:
#tot mom res for good events 

c = TCanvas("c", " ", 400, 400)
c.Divide(3,1)
ROOT.gStyle.SetOptStat(0)

max = 1
min = -1

h_res_nP_S = df_neutron.Filter("true_flag == 1").Histo1D(("h_res_nP_S", " neutron momentum residuals ", 100, min, max),"res_P_n")

ROOT.gPad.SetLeftMargin(0.15)
ROOT.gPad.SetRightMargin(0.04)
h_res_nP_S.GetXaxis().SetTitle("#Delta P/P")
h_res_nP_S.GetYaxis().SetTitle("Entries")
h_res_nP_S.Draw()


c.Draw()
c.SaveAs(Out_dir + "neutron_P_res_recogood_topologygood.pdf")

f_out.cd()
h_res_nP_S.GetValue().Write()

In [ ]:
# Energy res for good events
c = TCanvas("c", " ", 400, 400)
c.Divide(3,1)
ROOT.gStyle.SetOptStat(0)

max = 0.1
min = -0.1

h_res_nE_S = df_neutron.Filter("true_flag == 1").Histo1D(("h_res_nE_S", "neutron energy residuals (sig) ", 100, min, max),"res_E_n")

ROOT.gPad.SetLeftMargin(0.15)
ROOT.gPad.SetRightMargin(0.04)
h_res_nE_S.GetXaxis().SetTitle("#Delta E/E")
h_res_nE_S.GetYaxis().SetTitle("Entries")
h_res_nE_S.Draw()


c.Draw()
c.SaveAs(Out_dir + "neutron_E_res_recogood_topologygood.pdf")


f_out.cd()
h_res_nE_S.GetValue().Write()

In [ ]:
# Energy res for background events
c = TCanvas("c", " ", 400, 400)
c.Divide(3,1)
ROOT.gStyle.SetOptStat(0)

max = 0.6
min = -0.6

h_res_nE_B = df_neutron.Filter("true_flag == 0").Histo1D(("h_res_nE_B", "neutron energy residuals (bkg) ", 100, min, max),"res_E_n")

ROOT.gPad.SetLeftMargin(0.15)
ROOT.gPad.SetRightMargin(0.04)
h_res_nE_B.GetXaxis().SetTitle("#Delta E/E")
h_res_nE_B.GetYaxis().SetTitle("Entries")
h_res_nE_B.Draw()


c.Draw()
c.SaveAs(Out_dir + "neutron_E_res_recogood_topologybad.pdf")


f_out.cd()
h_res_nE_B.GetValue().Write()

In [ ]:
## Neutron momentum resolution components for background events
c = TCanvas("c", " ", 1500, 600)
c.Divide(3,1)
ROOT.gStyle.SetOptStat(0)

max = 2.5
min = -2.5

h_x1 = df_neutron.Filter("true_flag == 0").Histo1D(("h1", "  ", 100, min, max),"res_pn_x")
h_y1 = df_neutron.Filter("true_flag == 0").Histo1D(("h2", "  ", 100, min, max),"res_pn_y")
h_z1 = df_neutron.Filter("true_flag == 0").Histo1D(("h3", "  ", 100, min, max),"res_pn_z")

c.cd(1)
ROOT.gPad.SetLeftMargin(0.15)
ROOT.gPad.SetRightMargin(0.04)
h_x1.GetXaxis().SetTitle("#Delta Px/Px")
h_x1.GetYaxis().SetTitle("Entries")
h_x1.Draw()

c.cd(2)
ROOT.gPad.SetLeftMargin(0.15)
ROOT.gPad.SetRightMargin(0.04)
h_y1.GetXaxis().SetTitle("#Delta Py/Py")
h_y1.GetYaxis().SetTitle("Entries")

h_y1.Draw()

c.cd(3)
ROOT.gPad.SetLeftMargin(0.15)
ROOT.gPad.SetRightMargin(0.04)
h_z1.GetXaxis().SetTitle("#Delta Pz/Pz")
h_z1.GetYaxis().SetTitle("Entries")

h_z1.Draw()


c.Draw()
c.SaveAs(Out_dir + "neutron_p_res_components_recogood_topologybad.pdf")

f_out.cd()
h_x1.GetValue().Write()
h_y1.GetValue().Write()
h_z1.GetValue().Write()

In [ ]:
#tot mom res for background events
c = TCanvas("c", " ", 400, 400)
c.Divide(3,1)
ROOT.gStyle.SetOptStat(0)

max = 5
min = -1.5

h_res_nP_B = df_neutron.Filter("true_flag == 0").Histo1D(("h_res_nP_B", "neutron mometum residuals (bkg) ", 100, min, max),"res_P_n")

ROOT.gPad.SetLeftMargin(0.15)
ROOT.gPad.SetRightMargin(0.04)
h_res_nP_B.GetXaxis().SetTitle("#Delta P/P")
h_res_nP_B.GetYaxis().SetTitle("Entries")
h_res_nP_B.Draw()


c.Draw()
c.SaveAs(Out_dir + "neutron_P_res_recogood_topologybad.pdf")


f_out.cd()
h_res_nP_B.GetValue().Write()

In [ ]:
# Energy res for background events
c = TCanvas("c", " ", 500, 600)
c.Divide(3,1)
ROOT.gStyle.SetOptStat(0)

max = 1.5
min = -1.5

h_res_nE_B = df_neutron.Filter("true_flag == 0").Histo1D(("h_res_nE_B", " neutron enery res (bkg) ", 1000, min, max),"res_E_n")

ROOT.gPad.SetLeftMargin(0.15)
ROOT.gPad.SetRightMargin(0.04)
h_res_nE_B.GetXaxis().SetTitle("#Delta E/E")
h_res_nE_B.GetYaxis().SetTitle("Entries")
h_res_nE_B.Draw()


c.Draw()
c.SaveAs(Out_dir + "neutron_E_res_recogood_topologybad.pdf")

f_out.cd()
h_res_nE_B.GetValue().Write()

In [ ]:
### Clean up
del df_neutron

# Neutron kinematic-based prediction in space

In [ ]:
#x,y,z dist of the ECAL clusters

x_vecs = df.Take["ROOT::VecOps::RVec<double>"]("x_ecal")
y_vecs = df.Take["ROOT::VecOps::RVec<double>"]("y_ecal")
z_vecs = df.Take["ROOT::VecOps::RVec<double>"]("z_ecal")

x_all = np.concatenate([np.array(v, dtype=float) for v in x_vecs])
y_all = np.concatenate([np.array(v, dtype=float) for v in y_vecs])
z_all = np.concatenate([np.array(v, dtype=float) for v in z_vecs])


h_xy = ROOT.TH2F("h_xy_ECAL_Cl", " ", 100, -3000, 3000, 100, -6000, 1000)
h_zy = ROOT.TH2F("h_zy_ECAL_Cl", " ", 100, 20000, 28000, 100, -6000, 1000)

for x, y, z in zip(x_all, y_all, z_all):
    h_xy.Fill(x, y)
    h_zy.Fill(z, y)


c = TCanvas("c1", "Canvas", 1500, 600)
c.Divide(2, 1)
c.Draw()

ROOT.gStyle.SetOptStat(0)

c.cd(1)
pad1 = c.GetPad(1)
pad1.SetLeftMargin(0.15)   # margine sinistro più largo per la y
pad1.SetBottomMargin(0.15) # margine inferiore più largo per la x
h_xy.GetXaxis().SetTitle("x [mm]")
h_xy.GetYaxis().SetTitle("y [mm]")
h_xy.Draw("COLZ")

c.cd(2)
pad2 = c.GetPad(2)
pad2.SetLeftMargin(0.15)
pad2.SetBottomMargin(0.15)
h_zy.GetXaxis().SetTitle("z [mm]")
h_zy.GetYaxis().SetTitle("y [mm]")
h_zy.Draw("COLZ")

c.SaveAs(Out_dir + "ECAL_cluster_dist.pdf")

f_out.cd()
h_xy.Write()
h_zy.Write()

In [ ]:
df_true = df_flag.Filter("true_flag == 1")

x_vecs = df_true.Take["ROOT::VecOps::RVec<double>"]("x_ecal")
y_vecs = df_true.Take["ROOT::VecOps::RVec<double>"]("y_ecal")
z_vecs = df_true.Take["ROOT::VecOps::RVec<double>"]("z_ecal")

x_all = np.concatenate([np.array(v, dtype=float) for v in x_vecs])
y_all = np.concatenate([np.array(v, dtype=float) for v in y_vecs])
z_all = np.concatenate([np.array(v, dtype=float) for v in z_vecs])


h_xy = ROOT.TH2F("h_xy_ECAL_Cl_good_top", " ", 100, -3000, 3000, 100, -6000, 1000)
h_zy = ROOT.TH2F("h_zy_ECAL_Cl_good_top", " ", 100, 20000, 28000, 100, -6000, 1000)

for x, y, z in zip(x_all, y_all, z_all):
    h_xy.Fill(x, y)
    h_zy.Fill(z, y)


c = TCanvas("c1", "Canvas", 1500, 600)
c.Divide(2, 1)
c.Draw()

ROOT.gStyle.SetOptStat(0)

c.cd(1)
pad1 = c.GetPad(1)
pad1.SetLeftMargin(0.15)   # margine sinistro più largo per la y
pad1.SetBottomMargin(0.15) # margine inferiore più largo per la x
# h_xy.SetStats(0)
# h_zy.SetStats(0)
h_xy.GetXaxis().SetTitle("x [mm]")
h_xy.GetYaxis().SetTitle("y [mm]")
h_xy.Draw("COLZ")

c.cd(2)
pad2 = c.GetPad(2)
pad2.SetLeftMargin(0.15)
pad2.SetBottomMargin(0.15)
h_zy.GetXaxis().SetTitle("z [mm]")
h_zy.GetYaxis().SetTitle("y [mm]")
h_zy.Draw("COLZ")

c.SaveAs(Out_dir + "ECAL_cluster_dist_good_topology.pdf")
f_out.cd()
h_xy.Write()
h_zy.Write()

del df_true

In [ ]:
df_sel = df_flag.Filter("volume_flag == 1 && reco_flag == 1 && track_flag == 1")

x_vecs = df_sel.Take["ROOT::VecOps::RVec<double>"]("x_ecal")
y_vecs = df_sel.Take["ROOT::VecOps::RVec<double>"]("y_ecal")
z_vecs = df_sel.Take["ROOT::VecOps::RVec<double>"]("z_ecal")

x_all = np.concatenate([np.array(v, dtype=float) for v in x_vecs])
y_all = np.concatenate([np.array(v, dtype=float) for v in y_vecs])
z_all = np.concatenate([np.array(v, dtype=float) for v in z_vecs])


h_xy = ROOT.TH2F("h_xy_ECAL_Cl_selected", " ", 100, -3000, 3000, 100, -6000, 1000)
h_zy = ROOT.TH2F("h_zy_ECAL_Cl_selected", " ", 100, 20000, 28000, 100, -6000, 1000)

for x, y, z in zip(x_all, y_all, z_all):
    h_xy.Fill(x, y)
    h_zy.Fill(z, y)


c = TCanvas("c1", "Canvas", 1500, 600)
c.Divide(2, 1)
c.Draw()

ROOT.gStyle.SetOptStat(0)

c.cd(1)
pad1 = c.GetPad(1)
pad1.SetLeftMargin(0.15)   # margine sinistro più largo per la y
pad1.SetBottomMargin(0.15) # margine inferiore più largo per la x
# h_xy.SetStats(0)
# h_zy.SetStats(0)
h_xy.GetXaxis().SetTitle("x [mm]")
h_xy.GetYaxis().SetTitle("y [mm]")
h_xy.Draw("COLZ")

c.cd(2)
pad2 = c.GetPad(2)
pad2.SetLeftMargin(0.15)
pad2.SetBottomMargin(0.15)
h_zy.GetXaxis().SetTitle("z [mm]")
h_zy.GetYaxis().SetTitle("y [mm]")
h_zy.Draw("COLZ")

c.SaveAs(Out_dir + "ECAL_cluster_dist_selected.pdf")
f_out.cd()
h_xy.Write()
h_zy.Write()

del df_sel

In [ ]:
#predicted point in the edge of the ECAL

df_true = df_ECAL.Filter("true_flag == 1")

x_vecs = df_true.Take["ROOT::VecOps::RVec<double>"]("x_pred")
y_vecs = df_true.Take["ROOT::VecOps::RVec<double>"]("y_pred")
z_vecs = df_true.Take["ROOT::VecOps::RVec<double>"]("z_pred")

x_all = np.concatenate([np.array(v, dtype=float) for v in x_vecs])
y_all = np.concatenate([np.array(v, dtype=float) for v in y_vecs])
z_all = np.concatenate([np.array(v, dtype=float) for v in z_vecs])


h_xy = ROOT.TH2F("h_xy_ECAL_pred_goodtop", " ", 100, -3000, 3000, 100, -6000, 1000)
h_zy = ROOT.TH2F("h_zy_ECAL_pred_goodtop", " ", 100, 20000, 28000, 100, -6000, 1000)

for x, y, z in zip(x_all, y_all, z_all):
    h_xy.Fill(x, y)
    h_zy.Fill(z, y)


c = TCanvas("c1", "Canvas", 1500, 600)
c.Divide(2, 1)


ROOT.gStyle.SetOptStat(0)

c.cd(1)
pad1 = c.GetPad(1)
pad1.SetLeftMargin(0.15)   # margine sinistro più largo per la y
pad1.SetBottomMargin(0.15) # margine inferiore più largo per la x
# h_xy.SetStats(0)
# h_zy.SetStats(0)
h_xy.GetXaxis().SetTitle("x [mm]")
h_xy.GetYaxis().SetTitle("y [mm]")
h_xy.Draw("COLZ")

c.cd(2)
pad2 = c.GetPad(2)
pad2.SetLeftMargin(0.15)
pad2.SetBottomMargin(0.15)
h_zy.GetXaxis().SetTitle("z [mm]")
h_zy.GetYaxis().SetTitle("y [mm]")
h_zy.Draw("COLZ")

c.Draw()

c.SaveAs(Out_dir + "ECAL_cluster_pred_dist_goodtop.pdf")
f_out.cd()
h_xy.Write()
h_zy.Write()

del df_true

In [ ]:
df_nontrue = df_ECAL.Filter("true_flag == 0")

x_vecs = df_nontrue.Take["ROOT::VecOps::RVec<double>"]("x_pred")
y_vecs = df_nontrue.Take["ROOT::VecOps::RVec<double>"]("y_pred")
z_vecs = df_nontrue.Take["ROOT::VecOps::RVec<double>"]("z_pred")

x_all = np.concatenate([np.array(v, dtype=float) for v in x_vecs])
y_all = np.concatenate([np.array(v, dtype=float) for v in y_vecs])
z_all = np.concatenate([np.array(v, dtype=float) for v in z_vecs])


h_xy = ROOT.TH2F("h_xy_ECAL_pred_badtop", " ", 100, -3000, 3000, 100, -6000, 1000)
h_zy = ROOT.TH2F("h_zy_ECAL_pred_badtop", " ", 100, 20000, 28000, 100, -6000, 1000)

for x, y, z in zip(x_all, y_all, z_all):
    h_xy.Fill(x, y)
    h_zy.Fill(z, y)


c = TCanvas("c1", "Canvas", 1500, 600)
c.Divide(2, 1)


ROOT.gStyle.SetOptStat(0)

c.cd(1)
pad1 = c.GetPad(1)
pad1.SetLeftMargin(0.15)   # margine sinistro più largo per la y
pad1.SetBottomMargin(0.15) # margine inferiore più largo per la x
# h_xy.SetStats(0)
# h_zy.SetStats(0)
h_xy.GetXaxis().SetTitle("x [mm]")
h_xy.GetYaxis().SetTitle("y [mm]")
h_xy.Draw("COLZ")

c.cd(2)
pad2 = c.GetPad(2)
pad2.SetLeftMargin(0.15)
pad2.SetBottomMargin(0.15)
h_zy.GetXaxis().SetTitle("z [mm]")
h_zy.GetYaxis().SetTitle("y [mm]")
h_zy.Draw("COLZ")

c.Draw()

c.SaveAs(Out_dir + "ECAL_cluster_pred_dist_badtop.pdf")
f_out.cd()
h_xy.Write()
h_zy.Write()

del df_nontrue

In [ ]:
c = TCanvas("c", " ", 1500, 600)
ROOT.gStyle.SetOptStat(0)
c.Divide(3,1)

#max = 10000
#min = -10000


hx = df_ECAL.Filter("true_flag == 1").Histo1D(("h_pred_res_x_goodtop", " sig ", 100, -1000, 1000),"Res_seg_x")
hy = df_ECAL.Filter("true_flag == 1").Histo1D(("h_pred_res_y_goodtop", "  ", 100, -1000, 1000),"Res_seg_y")
hz = df_ECAL.Filter("true_flag == 1").Histo1D(("h_pred_res_z_goodtop", "  ", 100, -1000, 1000),"Res_seg_z")


ROOT.gPad.SetLeftMargin(0.15)
ROOT.gPad.SetRightMargin(0.04)

c.cd(1)
hx.GetXaxis().SetTitle("pred x - seg x")
hx.GetYaxis().SetTitle("Entries")
hx.Draw()


c.cd(2)
hy.GetXaxis().SetTitle("pred y - seg y")
hy.GetYaxis().SetTitle("Entries")
hy.Draw()

c.cd(3)
hz.GetXaxis().SetTitle("pred z - seg z")
hz.GetYaxis().SetTitle("Entries")
hz.Draw()

c.Draw()
c.SaveAs(Out_dir + "ECAL_cluster_pred_res_space_comp_goodtop.pdf")

f_out.cd()
hx.GetValue().Write()
hy.GetValue().Write()
hz.GetValue().Write()

In [ ]:
c = TCanvas("c", " ", 1500, 600)
ROOT.gStyle.SetOptStat(0)
c.Divide(3,1)

#max = 10000
#min = -10000


hx = df_ECAL.Filter("true_flag == 0").Histo1D(("h_pred_res_x_badtop", " bkg ", 100, -1000, 1000),"Res_seg_x")
hy = df_ECAL.Filter("true_flag == 0").Histo1D(("h_pred_res_y_badtop", "  ", 100, -1000, 1000),"Res_seg_y")
hz = df_ECAL.Filter("true_flag == 0").Histo1D(("h_pred_res_z_badtop", "  ", 100, -1000, 1000),"Res_seg_z")


ROOT.gPad.SetLeftMargin(0.15)
ROOT.gPad.SetRightMargin(0.04)

c.cd(1)
hx.GetXaxis().SetTitle("pred x - seg x")
hx.GetYaxis().SetTitle("Entries")
hx.Draw()


c.cd(2)
hy.GetXaxis().SetTitle("pred y - seg y")
hy.GetYaxis().SetTitle("Entries")
hy.Draw()

c.cd(3)
hz.GetXaxis().SetTitle("pred z - seg z")
hz.GetYaxis().SetTitle("Entries")
hz.Draw()

c.Draw()
c.SaveAs(Out_dir + "ECAL_cluster_pred_res_space_comp_badtop.pdf")

f_out.cd()
hx.GetValue().Write()
hy.GetValue().Write()
hz.GetValue().Write()

## ECAL Cluster resolution

In [ ]:
#### Cluster position residuals for good and bad events (no kinematic prediction)

c = TCanvas("c", " ", 1500, 600)
ROOT.gStyle.SetOptStat(0)
c.Divide(4,1)

#max = 10000
#min = -10000


hx = df_ECAL.Histo1D(("h_cl_res_x", "  ", 100, -200, 200),"Res_x")
hy = df_ECAL.Histo1D(("h_cl_res_y", "  ", 100, -200, 200),"Res_y")
hz = df_ECAL.Histo1D(("h_cl_res_z", "  ", 100, -200, 200),"Res_z")
ht = df_ECAL.Histo1D(("h_cl_res_t", "  ", 100, -100, 100),"Res_t")


ROOT.gPad.SetLeftMargin(0.15)
ROOT.gPad.SetRightMargin(0.04)

c.cd(1)
hx.GetXaxis().SetTitle("cluster x - ECAL hit x")
hx.GetYaxis().SetTitle("Entries")
hx.Draw()


c.cd(2)
hy.GetXaxis().SetTitle("cluster y - ECAL hit y")
hy.GetYaxis().SetTitle("Entries")
hy.Draw()

c.cd(3)
hz.GetXaxis().SetTitle("cluster z - ECAL hit z")
hz.GetYaxis().SetTitle("Entries")
hz.Draw()


c.cd(4)
ht.GetXaxis().SetTitle("cluster t - ECAL hit t")
ht.GetYaxis().SetTitle("Entries")
ht.Draw()

c.Draw()
c.SaveAs(Out_dir + "ECAL_cluster_res_space_time.pdf")

f_out.cd()
hx.GetValue().Write()
hy.GetValue().Write()
hz.GetValue().Write()
ht.GetValue().Write()

In [ ]:
c = TCanvas("c", " ", 1500, 600)
ROOT.gStyle.SetOptStat(0)
c.Divide(4,1)

#max = 10000
#min = -10000


hx = df_ECAL.Filter("true_flag == 1").Histo1D(("h_cl_res_x_goodtop", "  ", 100, -200, 200),"Res_x")
hy = df_ECAL.Filter("true_flag == 1").Histo1D(("h_cl_res_y_goodtop", "  ", 100, -200, 200),"Res_y")
hz = df_ECAL.Filter("true_flag == 1").Histo1D(("h_cl_res_z_goodtop", "  ", 100, -200, 200),"Res_z")
ht = df_ECAL.Filter("true_flag == 1").Histo1D(("h_cl_res_t_goodtop", "  ", 100, -100, 100),"Res_t")


ROOT.gPad.SetLeftMargin(0.15)
ROOT.gPad.SetRightMargin(0.04)

c.cd(1)
hx.GetXaxis().SetTitle("cluster x - tjpoint x")
hx.GetYaxis().SetTitle("Entries")
hx.Draw()


c.cd(2)
hy.GetXaxis().SetTitle("cluster y - tjpoint y")
hy.GetYaxis().SetTitle("Entries")
hy.Draw()

c.cd(3)
hz.GetXaxis().SetTitle("cluster z - tjpoint z")
hz.GetYaxis().SetTitle("Entries")
hz.Draw()


c.cd(4)
ht.GetXaxis().SetTitle("cluster t - tjpoint t")
ht.GetYaxis().SetTitle("Entries")
ht.Draw()

c.Draw()
c.SaveAs(Out_dir + "ECAL_cluster_res_space_time_goodtop.pdf")

f_out.cd()
hx.GetValue().Write()
hy.GetValue().Write()
hz.GetValue().Write()
ht.GetValue().Write()

In [ ]:
c = TCanvas("c", " ", 1500, 600)
ROOT.gStyle.SetOptStat(0)
c.Divide(4,1)

#max = 10000
#min = -10000


hx = df_ECAL.Filter("true_flag == 0").Histo1D(("h_cl_res_x_badtop", "  ", 100, -200, 200),"Res_x")
hy = df_ECAL.Filter("true_flag == 0").Histo1D(("h_cl_res_y_badtop", "  ", 100, -200, 200),"Res_y")
hz = df_ECAL.Filter("true_flag == 0").Histo1D(("h_cl_res_z_badtop", "  ", 100, -200, 200),"Res_z")
ht = df_ECAL.Filter("true_flag == 0").Histo1D(("h_cl_res_t_badtop", "  ", 100, -100, 100),"Res_t")


ROOT.gPad.SetLeftMargin(0.15)
ROOT.gPad.SetRightMargin(0.04)

c.cd(1)
hx.GetXaxis().SetTitle("cluster x - tjpoint x")
hx.GetYaxis().SetTitle("Entries")
hx.Draw()


c.cd(2)
hy.GetXaxis().SetTitle("cluster y - tjpoint y")
hy.GetYaxis().SetTitle("Entries")
hy.Draw()

c.cd(3)
hz.GetXaxis().SetTitle("cluster z - tjpoint z")
hz.GetYaxis().SetTitle("Entries")
hz.Draw()


c.cd(4)
ht.GetXaxis().SetTitle("cluster t - tjpoint t")
ht.GetYaxis().SetTitle("Entries")
ht.Draw()

c.Draw()
c.SaveAs(Out_dir + "ECAL_cluster_res_space_time_badtop.pdf")
f_out.cd()
hx.GetValue().Write()
hy.GetValue().Write()
hz.GetValue().Write()
ht.GetValue().Write()

## ECAL distance from prediction in space

In [ ]:
#ECAL distance

c = TCanvas("c", " ", 1500, 600)
ROOT.gStyle.SetOptStat(0)
c.Divide(2,1)

#max = 10000
#min = -10000


sig = df_ECAL.Filter("true_flag == 1").Histo1D(("h_dist_pred_cl_goodtop", " signal ", 100, -50, 1000),"distance_ecalpoint")
bkg = df_ECAL.Filter("true_flag == 0").Histo1D(("h_dist_pred_cl_badtop", "bkg  ", 100, -50, 1000),"distance_ecalpoint")



ROOT.gPad.SetLeftMargin(0.15)
ROOT.gPad.SetRightMargin(0.04)

c.cd(1)
sig.GetXaxis().SetTitle("distance between cilinder axis and nearest cl")
sig.GetYaxis().SetTitle("Entries")
sig.Draw()


c.cd(2)
bkg.GetXaxis().SetTitle("distance between cilineder axis and nearest cl")
bkg.GetYaxis().SetTitle("Entries")
bkg.Draw("")

c.Draw()
c.SaveAs(Out_dir + "ECAL_dist_pred_cl.pdf")
f_out.cd()
sig.GetValue().Write()
bkg.GetValue().Write()

In [ ]:
#ECAL distance

c = TCanvas("c", " ", 1500, 600)
ROOT.gStyle.SetOptStat(0)
c.Divide(2,1)

#max = 10000
#min = -10000

df_sel = df_flag.Filter("volume_flag == 1 && reco_flag == 1 && track_flag == 1")
sig = df_sel.Filter("true_flag == 1").Histo1D(("h_dist_pred_cl_gootvol_goodtrack_1track_goodtop", " signal distance  ", 100, -50, 1000),"distance_ecalpoint")
bkg = df_sel.Filter("true_flag == 0").Histo1D(("h_dist_pred_cl_gootvol_goodtrack_1track_badtop", "bkg distance   ", 100, -50, 1000),"distance_ecalpoint")



ROOT.gPad.SetLeftMargin(0.15)
ROOT.gPad.SetRightMargin(0.04)

c.cd(1)
sig.GetXaxis().SetTitle("distance between cilineder axis and nearest cl")
sig.GetYaxis().SetTitle("Entries")
sig.Draw()


c.cd(2)
bkg.GetXaxis().SetTitle("distance between cilineder axis and nearest cl")
bkg.GetYaxis().SetTitle("Entries")
bkg.Draw("")


c.Draw()
c.SaveAs(Out_dir + "ECAL_dist_pred_cl_selected.pdf")
f_out.cd()
sig.GetValue().Write()
bkg.GetValue().Write()

In [ ]:
c = TCanvas("c", " ", 600, 600)
ROOT.gStyle.SetOptStat(0)


ROOT.gPad.SetLeftMargin(0.15)
ROOT.gPad.SetRightMargin(0.04)

sig.GetXaxis().SetTitle("distance between cilineder axis and nearest cl")
sig.GetYaxis().SetTitle("Entries")
sig.SetLineColor(kRed)


bkg.GetXaxis().SetTitle("distance between cilineder axis and nearest cl")
bkg.GetYaxis().SetTitle("Entries")
bkg.Draw("")
sig.Draw("SAME")

c.Draw()
c.SaveAs(Out_dir + "ECAL_dist_pred_cl_selected_overlay.pdf")

In [ ]:
cN = TCanvas("c", " ", 600, 600)
ROOT.gStyle.SetOptStat(0)

ROOT.gPad.SetLeftMargin(0.15)
ROOT.gPad.SetRightMargin(0.04)

sig2 = df_sel.Filter("true_flag == 1 && distance_ecalpoint > 0.0").Histo1D(("sig", "  ", 100, -50, 1000),"distance_ecalpoint")
bkg2 = df_sel.Filter("true_flag == 0 && distance_ecalpoint > 0.0").Histo1D(("bkg", "  ", 100, -50, 1000),"distance_ecalpoint")

# Imposta titoli e colori
sig2.GetXaxis().SetTitle("distance between cylinder axis and nearest cluster [mm]")
sig2.GetYaxis().SetTitle("Normalized entries")
sig2.SetLineColor(ROOT.kRed)

bkg2.GetXaxis().SetTitle("distance between cylinder axis and nearest cluster [mm]")
bkg2.GetYaxis().SetTitle("Normalized entries")
bkg2.SetLineColor(ROOT.kBlue)

  
sig2.DrawNormalized("")
bkg2.DrawNormalized("SAME")

cN.Draw()
cN.SaveAs(Out_dir + "ECAL_dist_pred_cl_selected_overlay_normalized.pdf")

## Ratio between events with clusters within radius of predicted n pos in ECAL for sig/background

In [ ]:
number_f = df_ECAL.Filter("true_flag == 1 && distance_ecalpoint > 0.0 && distance_ecalpoint < 70.0").Count().GetValue()
number_tot = df_ECAL.Filter("true_flag == 1 && distance_ecalpoint > 0.0").Count().GetValue()

print("Fraction of signal events with distance < 70 mm:", number_f/number_tot)

In [ ]:
number_f = df_ECAL.Filter("true_flag == 0 && distance_ecalpoint > 0.0 && distance_ecalpoint < 70").Count().GetValue()
number_tot = df_ECAL.Filter("true_flag == 0 && distance_ecalpoint > 0.0").Count().GetValue()

print("Fraction of background events with distance < 70 mm:", number_f/number_tot)

## Distance between prediction and seg_det point

In [ ]:
c = TCanvas("c", " ", 1500, 600)
ROOT.gStyle.SetOptStat(0)
c.Divide(2,1)

#max = 10000
#min = -10000


sigSEG = df_ECAL.Filter("true_flag == 1").Histo1D(("h_dist_pred_segdet_goodtop", " signal ", 100, -50, 1000),"distance_segpoint")
bkgSEG = df_ECAL.Filter("true_flag == 0").Histo1D(("h_dist_pred_segdet_badtop", "bkg  ", 100, -50, 1000),"distance_segpoint")



ROOT.gPad.SetLeftMargin(0.15)
ROOT.gPad.SetRightMargin(0.04)

c.cd(1)
sigSEG.GetXaxis().SetTitle("distance between cilineder axis and nearest seg")
sigSEG.GetYaxis().SetTitle("Entries")
sigSEG.Draw()


c.cd(2)
bkgSEG.GetXaxis().SetTitle("distance between cilineder axis and nearest seg")
bkgSEG.GetYaxis().SetTitle("Entries")
bkgSEG.Draw("")

c.Draw()
c.SaveAs(Out_dir + "Seg_dist_pred_seg.pdf")
f_out.cd()
sigSEG.GetValue().Write()
bkgSEG.GetValue().Write()

In [ ]:
c = TCanvas("c", " ", 600, 600)
ROOT.gStyle.SetOptStat(0)


ROOT.gPad.SetLeftMargin(0.15)
ROOT.gPad.SetRightMargin(0.04)

sigSEG.GetXaxis().SetTitle("distance between cilineder axis and nearest seg ")
sigSEG.GetYaxis().SetTitle("Entries")
sigSEG.SetLineColor(kRed)


bkgSEG.GetXaxis().SetTitle("distance between cilineder axis and nearest seg ")
bkgSEG.GetYaxis().SetTitle("Entries")
bkgSEG.Draw("")
sigSEG.Draw("SAME")

c.Draw()
c.SaveAs(Out_dir + "Seg_dist_pred_seg_overlay.pdf")

### Ratio between events with seg_det within radius of predicted neutron pos in ECAL for sig/background

In [ ]:
###With additional selections
number_f = df_sel.Filter("true_flag == 1 && distance_segpoint > 0.0 && distance_segpoint < 25.0").Count().GetValue()
number_tot = df_sel.Filter("true_flag == 1 && distance_segpoint > 0.0").Count().GetValue()

print("Fraction of selected signal events:", number_f/number_tot)

In [ ]:
### Without additional selections
number_f = df_ECAL.Filter("true_flag == 1 && distance_segpoint > 0.0 && distance_segpoint < 25.0").Count().GetValue()
number_tot = df_ECAL.Filter("true_flag == 1 && distance_segpoint > 0.0").Count().GetValue()

print("Fraction of selected signal events without additional selections:", number_f/number_tot)

In [ ]:
### without additional selections
number_f = df_ECAL.Filter("true_flag == 0 && distance_segpoint > 0.0 && distance_segpoint < 25.0").Count().GetValue()
number_tot = df_ECAL.Filter("true_flag == 0 && distance_segpoint > 0.0").Count().GetValue()

print("Fraction of selected background events without additional selections:", number_f/number_tot)

## Neutron kinematic prediction in time

In [ ]:
df_time = df_flag.Filter("volume_flag == 1 && reco_flag == 1 && track_flag == 1 && space_flag == 1 && time_flag == 1")

In [ ]:
c = TCanvas("c", " ", 1500, 600)
ROOT.gStyle.SetOptStat(0)
c.Divide(2,1)

#max = 10000
#min = -10000


sig_T_seg = df_time.Filter("true_flag == 1").Histo1D(("h_t_res_seg_signal", " signal ", 100, -100, 25),"t_res_seg")
bkg_T_seg = df_time.Filter("true_flag == 0").Histo1D(("h_t_res_seg_bkg", "bkg  ", 100, -100, 25),"t_res_seg")



ROOT.gPad.SetLeftMargin(0.15)
ROOT.gPad.SetRightMargin(0.04)

c.cd(1)
sig_T_seg.GetXaxis().SetTitle(" (nearest seg pred T) - (nearest seg true T)")
sig_T_seg.GetYaxis().SetTitle("Entries")
sig_T_seg.Draw()


c.cd(2)
bkg_T_seg.GetXaxis().SetTitle("(nearest seg pred T) - (nearest seg true T)")
bkg_T_seg.GetYaxis().SetTitle("Entries")
bkg_T_seg.Draw("")

c.Draw()
c.SaveAs(Out_dir + "Seg_t_res.pdf")
f_out.cd()
sig_T_seg.GetValue().Write()
bkg_T_seg.GetValue().Write()

In [ ]:
cT = TCanvas("c", " ", 1500, 600)
ROOT.gStyle.SetOptStat(0)
cT.Divide(2,1)

#max = 10000
#min = -10000


sig_T_ecal = df_time.Filter("true_flag == 1").Histo1D(("h_t_res_ecal_signal", " signal ", 100, -25, 20),"t_res_ecal")
bkg_T_ecal = df_time.Filter("true_flag == 0").Histo1D(("h_t_res_ecal_bkg", "bkg  ", 100, -25, 20),"t_res_ecal")


ROOT.gPad.SetLeftMargin(0.15)
ROOT.gPad.SetRightMargin(0.04)

sig_T_ecal.SetLineColor(ROOT.kBlack)
bkg_T_ecal.SetLineColor(ROOT.kBlack)

cT.cd(1)
sig_T_ecal.GetXaxis().SetTitle(" #Delta t [ns]")
sig_T_ecal.GetYaxis().SetTitle("Entries")
sig_T_ecal.Draw()


cT.cd(2)
bkg_T_ecal.GetXaxis().SetTitle("#Delta t [ns]")
bkg_T_ecal.GetYaxis().SetTitle("Entries")
bkg_T_ecal.Draw("")

cT.Draw()
cT.SaveAs(Out_dir + "ECAL_t_res.pdf")
f_out.cd()
sig_T_ecal.GetValue().Write()
bkg_T_ecal.GetValue().Write()

# Closing the output file

In [ ]:
f_out.Close()